# Notebook 2: Advanced Quantum Tomography
## Deep Learning (DNN) & Variational Quantum Circuits (VQC) vs. Classical SVR

## 1. Scientific Context & Exploration Goals
In the previous notebook, we established a robust baseline using **Support Vector Regression (SVR)**, a classical kernel-based method. While effective, SVR relies on fixed kernels (RBF, Polynomial) which may not perfectly capture the complex geometry of quantum states under decoherence.

In this second exploratory phase, we move towards **Advanced Architectures**. Our goal is to determine if models with higher representational capacity (Deep Learning) or native quantum priors (VQC) can surpass the classical baseline, particularly in the regime of **impure states (decoherence)** and **limited data**.

We investigate two challengers:
1.  **Deep Neural Networks (DNN):** Using the universal approximation theorem to model the mapping from measurements to density matrices with high non-linearity.
2.  **Variational Quantum Circuits (VQC):** A "Quantum Machine Learning" approach. We hypothesize that a quantum circuit possesses a natural *inductive bias* for quantum data, potentially requiring fewer parameters to represent the Hilbert space than a classical network.

## 2. A Paradigm Shift: "Physics-Informed" Training
Unlike standard regression which minimizes the Euclidean distance (MSE), we introduce a **Custom Loss Function** grounded in Quantum Information Theory.

### The "Fidelity-Based" Backpropagation
Standard ML optimizes geometry. We want to optimize **physics**.
Instead of blindingly minimizing $MSE = ||\vec{r}_{pred} - \vec{r}_{real}||^2$, we configure our Neural Networks (both Classical and Quantum) to directly maximize the **Quantum Fidelity** ($F$).

During the **Backpropagation** pass, the gradient of the Fidelity is computed with respect to the model weights. This forces the optimizer to prioritize directions that increase the physical overlap between the predicted and true states.

**The Mathematical Loss Function:**
For a single qubit state defined by a Bloch vector $\vec{r}$, the loss $\mathcal{L}$ to minimize is:

$$\mathcal{L} = 1 - F(\rho_{pred}, \rho_{real})$$

Where the Fidelity $F$ for single-qubit Bloch vectors is given analytically by:
$$F(\vec{r}_{p}, \vec{r}_{t}) = \frac{1}{2} \left( 1 + \vec{r}_{p} \cdot \vec{r}_{t} + \sqrt{(1 - ||\vec{r}_{p}||^2)(1 - ||\vec{r}_{t}||^2)} \right)$$

* **Interpretation:** The term $\vec{r}_{p} \cdot \vec{r}_{t}$ aligns the vectors directionally. The term under the square root penalizes errors in **purity** (vector length). This allows the DNN to specifically "learn" decoherence.

## 3. Architecture Overview

### A. Deep Neural Network (DNN - PyTorch)
* **Structure:** A Multi-Layer Perceptron (MLP) with fully connected layers and non-linear activation functions (ReLU).
* **Why:** To test if a "Universal Approximator" can learn the noise models better than fixed kernels.

### B. Variational Quantum Circuit (VQC - PennyLane)
* **Concept:** We use a parameterized quantum circuit as the model.
* **Mechanism:**
    1.  **Encoding:** Classical inputs ($X, Y, Z$) are embedded into a quantum state via rotation gates.
    2.  **Processing:** A sequence of trainable gates (Ansatz) manipulates the state.
    3.  **Measurement:** We measure the expectation values of Pauli operators to obtain the output vector.
* **Hypothesis:** "Quantum for Quantum". A quantum circuit naturally evolves on the Bloch sphere (or inside it for mixed states via subsystems), which might offer better generalization with fewer parameters.

## 4. Implementation: High-Performance Computing (GPU)
With SVC, we had quite some long training time. So in order to handle the computational load of training deep networks and simulating quantum circuits, we leverage **GPU Acceleration**:
* **PyTorch (CUDA/MPS):** For tensor operations and automatic differentiation of the DNN.
* **PennyLane Lightning GPU:** Using high-performance state-vector simulators (like `lightning.gpu` or `lightning.qubit`) to accelerate the VQC simulation and gradient calculation (adjoint differentiation).

We also do this as a way to learn modern high-performance ML pipelines.

## 5. Input/Output Interfaces
To ensure a rigorous comparison with the SVR baseline from Notebook 1, the I/O structure remains identical:

* **Input $\mathbf{X}$:** Noisy measurement expectations $[ \langle X \rangle_{noise}, \langle Y \rangle_{noise}, \langle Z \rangle_{noise} ]$.
* **Output $\mathbf{y}$:** Predicted Bloch vector components $[\hat{x}, \hat{y}, \hat{z}]$.

*Note: The predicted vector is implicitly constrained to valid physical states (norm $\le$ 1) either via activation functions (Tanh) or penalty terms in the loss.*

In [ ]:
import torch
import torch.nn as nn
import pennylane as qml

# ==========================================
# 1. SETUP GPU & DEVICE AGNOSTIC CODE
# ==========================================
def get_device():
    """Détecte automatiquement le meilleur accélérateur disponible."""
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        # Pour les Mac M1/M2/M3 (Metal Performance Shaders)
        return torch.device("mps")
    else:
        return torch.device("cpu")

DEVICE = get_device()
print(f"✅ Computation Device: {DEVICE}")

# ==========================================
# 2. CUSTOM LOSS : QUANTUM FIDELITY
# ==========================================
class QuantumFidelityLoss(nn.Module):
    """
    Fonction de perte inversée : Loss = 1 - Fidélité.
    Pousse le réseau à maximiser la superposition physique avec l'état cible.
    """
    def __init__(self):
        super(QuantumFidelityLoss, self).__init__()

    def forward(self, y_pred, y_true):
        # y_pred, y_true shape: (batch_size, 3) correspondant à [X, Y, Z]
        
        # 1. Calcul des normes carrées (r^2)
        # On clippe à 1.0 - epsilon pour éviter les racines de nombres négatifs
        # si le réseau prédit temporairement un vecteur > 1.
        r2_pred = torch.sum(y_pred**2, dim=1).clamp(max=1.0 - 1e-6)
        r2_true = torch.sum(y_true**2, dim=1).clamp(max=1.0 - 1e-6)
        
        # 2. Produit scalaire (Orientation)
        dot_prod = torch.sum(y_pred * y_true, dim=1)
        
        # 3. Terme de pureté (Grandeur)
        # Formule : sqrt((1 - r_pred^2)(1 - r_true^2))
        purity_term = torch.sqrt((1.0 - r2_pred) * (1.0 - r2_true))
        
        # 4. Fidélité
        fidelity = 0.5 * (1.0 + dot_prod + purity_term)
        
        # 5. On retourne la perte moyenne (on veut minimiser 1 - F)
        return (1.0 - fidelity).mean()

# ==========================================
# 3. MODEL 1: DEEP NEURAL NETWORK (DNN)
# ==========================================
class TomographyDNN(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, output_dim=3):
        super(TomographyDNN, self).__init__()
        
        self.net = nn.Sequential(
            # Entrée : X_mean, Y_mean, Z_mean
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            
            # Sortie : X_pred, Y_pred, Z_pred
            nn.Linear(hidden_dim, output_dim),
            
            # Tanh force la sortie entre [-1, 1], ce qui aide physiquement
            # car une coordonnée de Bloch ne peut pas dépasser 1.
            nn.Tanh() 
        )

    def forward(self, x):
        return self.net(x)

# ==========================================
# 4. MODEL 2: VARIATIONAL QUANTUM CIRCUIT (VQC)
# ==========================================
# Configuration du simulateur quantique
# 'lightning.qubit' est un simulateur C++ rapide pour CPU.
# Pour le GPU 'vrai', il faut 'lightning.gpu' (voir explications plus bas).
n_qubits = 4
dev = qml.device("lightning.qubit", wires=n_qubits) 

@qml.qnode(dev, interface="torch", diff_method="adjoint")
def quantum_circuit(inputs, weights):
    """
    Circuit Variationnel.
    inputs: Données classiques (Batch)
    weights: Paramètres apprenables du circuit
    """
    # 1. Encodage des données (Angle Embedding)
    # On encode les features X, Y, Z dans les rotations des qubits
    # On répète les inputs pour remplir les 4 qubits si nécessaire
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')
    
    # 2. Couches Variationnelles (Ansatz)
    # StrongEntanglingLayers est très expressif pour apprendre des états complexes
    qml.StrongEntanglingLayers(weights, wires=range(n_qubits))
    
    # 3. Mesures : On veut 3 sorties (X, Y, Z)
    # On mesure les espérances de Pauli sur les 3 premiers qubits pour récupérer 3 valeurs.
    return [qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2))]

class TomographyVQC(nn.Module):
    def __init__(self, n_layers=3):
        super(TomographyVQC, self).__init__()
        
        # Définition de la forme des poids pour StrongEntanglingLayers
        weight_shapes = {"weights": (n_layers, n_qubits, 3)}
        
        # qml.qnn.TorchLayer transforme le QNode PennyLane en une couche PyTorch standard !
        self.q_layer = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)
        
        # Optionnel : Une petite couche classique pour "calibrer" la sortie quantique
        # Cela aide le VQC à mapper exactement vers l'espace de Bloch cible.
        self.post_processing = nn.Linear(3, 3) 

    def forward(self, x):
        # On passe dans le quantique
        x_q = self.q_layer(x)
        # On ajuste légèrement l'échelle
        return torch.tanh(self.post_processing(x_q))

NameError: name 'null' is not defined